#Applied Natural Language Processing 955G5
##Computer Based Examination, 2023

Remember, you can add cells and change their type (between code and text/markdown) as required to answer the questions.

In [ ]:
# update your candidate number here
candidate_number = 251038

# Question 1 (50 marks)

This question is about document similarity and information retrieval.

In [1]:
### do not change the code in this cell
# make sure you run this cell
from math import log, sqrt
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import nltk
nltk.download("punkt")
nltk.download("stopwords")
stops = set(stopwords.words('english'))

documents=[["NLTK", ",", "or", "Natural", "Language", "Toolkit" ",", "is", "a", "Python", "package", "that", "you", "can", "use", "for", "NLP", "."],
           ["Neuro-linguistic", "programming", "(", "NLP", ")", "is", "a", "pseudoscientific", "approach", "to", "communication" ",", "personal", "development", "and", "psychotherapy", "."],
           ["Python", "is", "a", "genus", "of", "constricting", "snakes", "in", "the", "Pythonidae", "family", "."]]


query="NLP Python"

[nltk_data] Downloading package punkt to C:\Users\Ghadah Al-
[nltk_data]     Chaab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ghadah Al-
[nltk_data]     Chaab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


a) This question deals with the similarity of documents in terms of comparing the words they contain. Describe two approaches to measuring the similarity of individual words. (4 marks)

There are two methods that are frequently used to measuring the similarity of individual words:

Cosine Similarity: This approach represents words as vectors in a high-dimensional space, where each dimension corresponds to a word in the vocabulary. The similarity is calculated based on the cosine of the angle between the two word vectors. It takes into account the frequency or occurrence of words in a document. Cosine similarity ranges from 0 to 1, with 1 indicating perfect similarity. However, in the case of individual words, cosine similarity may not be as informative as it is for comparing documents or larger text segments.


Similar to Jaccard, this method interprets words as collections of symbols or tokens. By dividing the size of the intersection of the two sets (common characters or tokens) by the size of their union, it determines how similar the two sets are. The Jaccard similarity, for instance, would be 5/6 if we compared the words "colour" and "colour," since they share all characters bar the "u" in "colour."

b)
i) Write a function that takes a set of tokenized documents as input and returns the same documents after removing stopwords and punctuation, and applying case normalisation to the remaining tokens. So for example, the documents `[["This", "is", "Alice", "'s", "document", "."], ["Bob", "'s", "document", "."]]` would become `[["alice", "document"], ["bob", "document"]]`. Apply this function to the list `documents` to produce a list called `normalised_documents`. (10 marks)

In [2]:
import string
import nltk
import re
from nltk.corpus import stopwords

nltk.download("stopwords")

def preprocessing(text):
    stops = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    
    def preprocess(word):
        word = re.sub(r"(\w)'s", r"\1", word)  # to remove possessive 's
        word = word.lower().strip(string.punctuation)
        if word not in stops and word != "":
            return word
        return None
    
    return [[preprocess(word) for word in t if preprocess(word) is not None] for t in text]

tokenized_documents = documents
normalised_documents = preprocessing(tokenized_documents)

print(normalised_documents)



[['nltk', 'natural', 'language', 'toolkit', 'python', 'package', 'use', 'nlp'], ['neuro-linguistic', 'programming', 'nlp', 'pseudoscientific', 'approach', 'communication', 'personal', 'development', 'psychotherapy'], ['python', 'genus', 'constricting', 'snakes', 'pythonidae', 'family']]


[nltk_data] Downloading package stopwords to C:\Users\Ghadah Al-
[nltk_data]     Chaab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ii) What impact will stopword removal and case normalisation have on the similarity of documents? (4 marks)

Stopword removal: Stopwords, such as "the," "is," "and," etc., are often used words in a language that don't have much significance. Eliminating stopwords from a manuscript can assist readers focus on the most crucial words and lessen background noise.By removing stopwords, the similarity of documents may be affected because the frequency and distribution of stopwords can differ between documents. Removing stopwords can also help to highlight the unique content words that contribute to the overall meaning of the document.

Case normalisation involves changing all words to either lowercase letters or uppercase letters. By using this method, the effect of letter case on word similarity may be reduced. After case normalisation, "apple" and "Apple," for instance, would be considered to be the same word. Case normalization ensures that words with the same letters but different cases are viewed as equivalent, which increases the similarity of documents.

iii) Write a function that takes a list of tokenized documents as input and produces a list of bag-of-words representations. That is, each document should be represented as a FreqDist dictionary mapping tokens to the count of each token in the document. For example, the document `["alice", "document"]` would become `FreqDist({'alice': 1, 'document':1})`. Apply this function to the list `normalised_documents` to produce a list called `bags`. (6 marks)

In [3]:
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords

# Create bags of words
def bag_of_words(documents):
    bags = [FreqDist(doc) for doc in documents]
    return bags

bags = bag_of_words(normalised_documents)

# Print word frequencies for each document's bag-of-words representation
for bag in bags:
    for word, frequency in bag.items():
        print(f"{word}: {frequency}", end=", ")
print()


nltk: 1, natural: 1, language: 1, toolkit: 1, python: 1, package: 1, use: 1, nlp: 1, neuro-linguistic: 1, programming: 1, nlp: 1, pseudoscientific: 1, approach: 1, communication: 1, personal: 1, development: 1, psychotherapy: 1, python: 1, genus: 1, constricting: 1, snakes: 1, pythonidae: 1, family: 1, 


iv) Given the definition below, write a function that takes a list of bag-of-words document representations as input and calculates inverse document frequencies. The output of the function should be dictionary that has tokens as keys and inverse document frequencies as values. 

$$ idf(t) = \log_2 \left( \frac{N}{df(t)} \right) $$

Here, $df(t)$, is the document frequency of the token, $t$, i.e. the number of documents it occurs in, and $N$ is the total number of documents. For example, if alice occurs in 1 out of 2 documents, then the inverse document frequency dictionary should return the value $1 = \log_2(2)$ for the key `'alice'`.

Apply this function to the list `bags` to produce a dictionary called `idf`.

(4 marks)

In [4]:
import math

def calculate_idf(bags):
    N = len(bags)
    df = {}
    
    # find the frequency for each token
    for bag in bags:
        for token in bag.keys():
            df[token] = df.get(token, 0) + 1
    
    idf = {token: math.log2(N / df_token) for token, df_token in df.items()}
    return idf

# apply the function to the list of bags
idf = calculate_idf(bags)

# Print the inverse document frequency dictionary
print(idf)

{'nltk': 1.584962500721156, 'natural': 1.584962500721156, 'language': 1.584962500721156, 'toolkit': 1.584962500721156, 'python': 0.5849625007211562, 'package': 1.584962500721156, 'use': 1.584962500721156, 'nlp': 0.5849625007211562, 'neuro-linguistic': 1.584962500721156, 'programming': 1.584962500721156, 'pseudoscientific': 1.584962500721156, 'approach': 1.584962500721156, 'communication': 1.584962500721156, 'personal': 1.584962500721156, 'development': 1.584962500721156, 'psychotherapy': 1.584962500721156, 'genus': 1.584962500721156, 'constricting': 1.584962500721156, 'snakes': 1.584962500721156, 'pythonidae': 1.584962500721156, 'family': 1.584962500721156}


v) Write a function that takes a list of bag-of-words document representations and a dictionary of inverse document frequencies and produces a new set of $tfidf$ representations. Apply this function to the list `bags` and the dictionary `idf` to produce a list of document representations called `tfidf_reps`. (4 marks)

In [5]:
def calculate_tfidf(bags, idf_dict):
    tfidf_reps = []
    
    for bag in bags:
        tfidf_rep = {token: tf * idf_dict[token] for token, tf in bag.items()}
        tfidf_reps.append(tfidf_rep)
    
    return tfidf_reps

# Apply the function to the list of bags and the idf dictionary
tfidf_reps = calculate_tfidf(bags, idf)

# Print the TF-IDF representations
for doc, tfidf_rep in zip(normalised_documents, tfidf_reps):
    print(f"Document: {doc} => TF-IDF Representation: {tfidf_rep}")


Document: ['nltk', 'natural', 'language', 'toolkit', 'python', 'package', 'use', 'nlp'] => TF-IDF Representation: {'nltk': 1.584962500721156, 'natural': 1.584962500721156, 'language': 1.584962500721156, 'toolkit': 1.584962500721156, 'python': 0.5849625007211562, 'package': 1.584962500721156, 'use': 1.584962500721156, 'nlp': 0.5849625007211562}
Document: ['neuro-linguistic', 'programming', 'nlp', 'pseudoscientific', 'approach', 'communication', 'personal', 'development', 'psychotherapy'] => TF-IDF Representation: {'neuro-linguistic': 1.584962500721156, 'programming': 1.584962500721156, 'nlp': 0.5849625007211562, 'pseudoscientific': 1.584962500721156, 'approach': 1.584962500721156, 'communication': 1.584962500721156, 'personal': 1.584962500721156, 'development': 1.584962500721156, 'psychotherapy': 1.584962500721156}
Document: ['python', 'genus', 'constricting', 'snakes', 'pythonidae', 'family'] => TF-IDF Representation: {'python': 0.5849625007211562, 'genus': 1.584962500721156, 'constric

c)
i) Write a function that takes a pair of bag-of-word representations and calculates their dot product. (4 marks)

In [9]:
def dot_product(bag1, bag2):
    dot_product = sum(bag1.get(token, 0) * bag2.get(token, 0) for token in set(bag1) & set(bag2))
    return dot_product
# Calculate and print dot products for each pair of bag-of-words representations
for i in range(len(bags)):
    for j in range(i + 1, len(bags)):
        dotproduct = dot_product(bags[i], bags[j])
        print(f"Dot Product between Document {i + 1} and Document {j + 1}: {dotproduct}")


Dot Product between Document 1 and Document 2: 1
Dot Product between Document 1 and Document 3: 1
Dot Product between Document 2 and Document 3: 0


ii) Write a function that takes a pair of vectors and calculates their cosine similarity. (4 marks)

In [11]:
import math

def calculate_cosine_similarity(vector1, vector2):
    dot_product = sum(vector1.get(token, 0) * vector2.get(token, 0) for token in set(vector1) & set(vector2))
    magnitude1 = math.sqrt(sum(val ** 2 for val in vector1.values()))
    magnitude2 = math.sqrt(sum(val ** 2 for val in vector2.values()))
    
    if magnitude1 != 0 and magnitude2 != 0:
        cosine_similarity = dot_product / (magnitude1 * magnitude2)
        return cosine_similarity
    else:
        return 0.0  # If one of the vectors is the zero vector, similarity is 0
# Calculate and print cosine similarities for each pair of bag-of-words representations
for i in range(len(bags)):
    for j in range(i + 1, len(bags)):
        cosine_similarity = calculate_cosine_similarity(bags[i], bags[j])
        print(f"Cosine Similarity between Document {i + 1} and Document {j + 1}: {cosine_similarity}")


Cosine Similarity between Document 1 and Document 2: 0.1178511301977579
Cosine Similarity between Document 1 and Document 3: 0.14433756729740646
Cosine Similarity between Document 2 and Document 3: 0.0


iii) Put the string in `query` through the appropriate pre-processing to produce a tfidf bag-of-words representation, and find the index of the most similar document. Print out the corresponding document from the list `documents`. (10 marks)

In [16]:
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import math

nltk.download("punkt")
nltk.download("stopwords")

# Preprocessing function
def preprocessing(text):
    stops = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    
    def preprocess(word):
        word = re.sub(r"(\w)'s", r"\1", word)  # to remove possessive 's
        word = word.lower().strip(string.punctuation)
        if word not in stops and word != "":
            return word
        return None
    
    return [[preprocess(word) for word in t if preprocess(word) is not None] for t in text]

# Tokenized documents
documents = [
    ["NLTK", ",", "or", "Natural", "Language", "Toolkit", ",", "is", "a", "Python", "package", "that", "you", "can", "use", "for", "NLP", "."],
    ["Neuro-linguistic", "programming", "(", "NLP", ")", "is", "a", "pseudoscientific", "approach", "to", "communication", ",", "personal", "development", "and", "psychotherapy", "."],
    ["Python", "is", "a", "genus", "of", "constricting", "snakes", "in", "the", "Pythonidae", "family", "."]
]

# Apply preprocessing to the documents
tokenized_documents = documents
normalised_documents = preprocessing(tokenized_documents)

# Calculate bags of words from the normalized documents
bags = [FreqDist(doc) for doc in normalised_documents]

# Query string
query = "NLTK is a Python package for NLP."

# Preprocess the query string and convert it to a bag-of-words representation
query_words = nltk.word_tokenize(query)
preprocessed_query = [preprocess(word) for word in query_words if preprocess(word) is not None]
query_bag = FreqDist(preprocessed_query)

# Calculate TF-IDF representation for the query
query_tfidf = {token: tf * idf.get(token, 0) for token, tf in query_bag.items()}

# Calculate cosine similarity between query and document bags
similarities = [calculate_cosine_similarity(query_tfidf, bag) for bag in bags]

# Find the index of the most similar document
most_similar_index = similarities.index(max(similarities))

# Print the corresponding document
print(f"Most Similar Document:\n{documents[most_similar_index]}")


Most Similar Document:
['NLTK', ',', 'or', 'Natural', 'Language', 'Toolkit', ',', 'is', 'a', 'Python', 'package', 'that', 'you', 'can', 'use', 'for', 'NLP', '.']


[nltk_data] Downloading package punkt to C:\Users\Ghadah Al-
[nltk_data]     Chaab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ghadah Al-
[nltk_data]     Chaab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
